In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install transformers
!pip install multiprocess
!pip install tensorboardX
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path '/content/pyrouge/tools/ROUGE-1.5.5'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/bheinzerling/pyrouge/archive/master.zip
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: pyrouge
Version: 0.1.3
Summary: A Python wrapper for the ROUGE summarization evaluation package.
Home-page: https://github.com/noutenki/pyrouge
Author: Benjamin Heinzerling, Anders Johannsen
Author-email: benjamin.heinzerling@h-its.org
License: LICENSE.txt
Location: /usr/local/lib/python3.8/dist-packages
Requires: 
Required-by: 
fatal: destination path 'pyrouge' already exists and is not an empty directory.
2023-03-02 06:32:05,667 [MainThread  ] [INFO ]  Set ROUGE home directory to /content/pyrouge/tools/ROUGE-1.5.5.


In [18]:
import os
import json
from tqdm import tqdm
import re
import easydict
from rouge import Rouge

In [19]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid'


In [20]:
def data_load(DATAPATH):
    filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
    filenames.sort()
    filenames
    list_dic = []

    for file in filenames:
      filelocation = os.path.join(DATAPATH, file)

      with open(filelocation, 'r') as json_file:
        data = json.load(json_file)['documents']

        for x in tqdm (range(len(data))):
          text = data[x]['text']
          text = str(text).replace('"', "'")

          extractive = data[x]['extractive']
          for index, value in enumerate(extractive):
            if value == None:
              extractive[index] = 0

          p = re.compile('(?<=sentence\'\: \')(.*?)(?=\'highlight_indices)')
          texts = p.findall(text)

          sentences = []
          for t in texts:
            sentence = t[:-3]
            sentences.append(sentence)

          mydict = {}
          mydict['text'] = sentences
          mydict['extractive'] = extractive
          list_dic.append(mydict)

    return list_dic

In [21]:
list_dic = data_load(DATAPATH)

100%|██████████| 7008/7008 [00:00<00:00, 10704.78it/s]


## 원본 본문 및 요약 추출

In [7]:
selected_texts = []
for j in tqdm(range(len(list_dic))):
    selected_texts.append([list_dic[j]['text'][i] for i in list_dic[j]['extractive']])


100%|██████████| 40134/40134 [00:00<00:00, 406681.94it/s]


In [8]:
input_text2 = []
for i in tqdm(range(len(list_dic))):
    input_text2.append(list_dic[i]['text'])

100%|██████████| 40134/40134 [00:00<00:00, 1134893.39it/s]


In [9]:
len(input_text2)

40134

In [10]:
input_text2[0]

['[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,',
 '처분 등이 있은 날부터 1년을 경과하면',
 '제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.',
 '[2] 일반적으로 행정처분에 효력기간이 정하여져 있는 경우에는 그 기간의 경과로 그 행정처분의 효력은 상실되며,',
 '다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.',
 '[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,',
 '이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.']

In [22]:
selected_texts = []
for j in tqdm(range(len(list_dic))):
    selected_texts.append([list_dic[j]['text'][i] for i in list_dic[j]['extractive']])


100%|██████████| 40134/40134 [00:00<00:00, 279121.84it/s]


In [23]:
selected_texts[0]

['[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,',
 '다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.',
 '이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.']

## 예측 추출요약

In [11]:
import sys 
sys.path.append("/content/drive/MyDrive/인공지능/추출요약")
from SRC.train import new_inference

In [12]:
new_inference

<function SRC.train.new_inference(input_data, test_from, encoder, visible_gpus, gpu_ranks, world_size)>

In [13]:
test_from = "/content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer_result/model_step_50000.pt"

In [14]:
test = new_inference(input_text2[0], test_from, "transformer", "0", "0",1)

In [16]:
print(test)

['제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.', '[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,', '[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,']


## 평가

In [51]:
def rouge_f_r_summary(hypotheses, references):
    rouge = Rouge()

    reference = ' '.join(references)
    hypothese = ' '.join(hypotheses)

    scores = rouge.get_scores(hypothese, reference, avg=True)
    rouge_f_1 = scores['rouge-1']['f']
    rouge_r_1 = scores['rouge-1']['r']
    rouge_f_2 = scores['rouge-2']['f']
    rouge_r_2 = scores['rouge-2']['r']
    rouge_f_L = scores['rouge-l']['f']
    rouge_r_L = scores['rouge-l']['r']
    return rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L

In [52]:
rouge_f_1, rouge_f_L, rouge_f_2, rouge_r_1, rouge_r_2, rouge_r_L= rouge_f_r_summary(test, selected_texts[0])
print("--------------------------Rouge-F---------------------------")
print("Rouge-F_1:", rouge_f_1)
print("Rouge-F_2:", rouge_f_2)
print("Rouge-F_L:", rouge_f_L)
print("")
print("--------------------------Rouge-R---------------------------")
print("Rouge-R_1:", rouge_r_1)
print("Rouge-R_2:", rouge_r_2)
print("Rouge-R_L:", rouge_r_L)


--------------------------Rouge-F---------------------------
Rouge-F_1: 0.21568626951172637
Rouge-F_2: 0.1834862335527314
Rouge-F_L: 0.21568626951172637

--------------------------Rouge-R---------------------------
Rouge-R_1: 0.21153846153846154
Rouge-R_2: 0.1724137931034483
Rouge-R_L: 0.21153846153846154
